<a href="https://colab.research.google.com/github/wesley28/ProcessamentoDigitalDeImagens/blob/main/08)_Separa%C3%A7%C3%A3o_de_canais_de_cores_em_Imagens_Coloridas_(tipo_PGM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
import math
from collections import Counter


class ImagemBPM:

  def __init__(self, tipo='', largura=0, altura=0, valor_maximo = 1, colorido=False):
    self.tipo = tipo
    self.largura = largura
    self.altura = altura
    self.valor_maximo = valor_maximo
    self.colorido = colorido


  def _read(self, path):
    return open(path, "r").read().split('\n')


  def extrair_imagem(self, path, default_rgb = 0):
    ls_imagem = self._read(path)

    if self.colorido:
      self.tipo = ls_imagem[0]
      self.largura = int(ls_imagem[2].split()[0])
      self.altura = int(ls_imagem[2].split()[1])
      self.valor_maximo = int(ls_imagem[3])

      conteudo = ls_imagem[5:]
      conteudo = [i.split(' ') for i in conteudo]
      np_conteudo = np.array(conteudo)
      np_conteudo = np.concatenate(np_conteudo)
      np_conteudo = np.delete(np_conteudo, np.where(np_conteudo == ''))
      self.conteudo = [int(i) for i in np_conteudo]

      lista_rgb = {
          'red': np.array([]),
          'green': np.array([]),
          'blue': np.array([])
      }

      lista_rgb['red'] = np.array([self.conteudo[i] if i % 3 == 0 else default_rgb for i in range(len(self.conteudo))])
      lista_rgb['green'] = np.array([self.conteudo[i] if i % 3 == 1 else default_rgb for i in range(len(self.conteudo))])
      lista_rgb['blue'] = np.array([self.conteudo[i] if i % 3 == 2 else default_rgb for i in range(len(self.conteudo))])
      self.lista_rgb = lista_rgb

    else:
      self.tipo = ls_imagem[0]
      self.largura = int(ls_imagem[1].split()[0])
      self.altura = int(ls_imagem[1].split()[1])
      self.valor_maximo = int(ls_imagem[2])
      self.conteudo = np.array([int(i) if i != '' else 0 for i in ls_imagem[2 : self.altura * self.largura + 2]])


  def compactar(self, fator):
    assert self.altura % fator == 0, 'O fator de compactação precisa ser múltiplo da altura'
    assert self.largura % fator == 0, 'O fator de compactação precisa ser múltiplo da largura'

    self.fator = fator
    self.nova_altura = int(self.altura / self.fator)
    self.nova_largura = int(self.largura / self.fator)

    rs_conteudo = self.conteudo.reshape(int((self.altura * self.largura) / (self.fator * self.fator)), self.fator * self.fator)

    conteudo_compactado = []
    for agrupamento in self._gen_agrupamento_pixels(rs_conteudo):
      conteudo_compactado.append(agrupamento)

    self.conteudo_compactado = conteudo_compactado


  def gerar_media_pixels_rgb(self):
    self.gerar_media_pixels()
    conteudo_media_rgb = np.array(range(self.conteudo_agrupado.shape[0] * 3))
    for i in range(len(self.conteudo_agrupado)):
      conteudo_media_rgb[3 * i: 3 * i + 3] = self.conteudo_agrupado[i]

    self.conteudo_media_rgb = conteudo_media_rgb


  def gerar_media_pixels(self, escala_cinza=False):
    np_array = np.array(self.conteudo)
    np_array_reshape = np_array.reshape(np_array.shape[0] // 3, 3)
    conteudo_agrupado = []
    for agrupamento in self._gen_agrupamento_pixels(np_array_reshape):
      conteudo_agrupado.append(agrupamento)

    self.conteudo_agrupado = np.array(conteudo_agrupado)

    if escala_cinza:
      self.tipo = 'P2'


  def _gen_agrupamento_pixels(self, ls_conteudo):
    for valores in ls_conteudo:
      #yield max(valores)
      #yield min(valores)
      yield round(sum(valores) / len(valores))
      #c = Counter(valores)
      #yield c.most_common(1)[0][0]


  def redimensionar(self, nv_largura, nv_altura):
    self.nv_largura = nv_largura
    self.nv_altura = nv_altura
    matriz_conteudo = self.conteudo.reshape(self.largura, self.altura)
    matriz_conteudo_copia = matriz_conteudo.copy()

    diff_largura = nv_largura - self.largura
    if diff_largura != 0:
      lt_linhas = list(range(self.largura))
      random.shuffle(lt_linhas)

      if diff_largura > 0:
        linhas_add = lt_linhas[:diff_largura]
        for id in linhas_add:
          matriz_conteudo_copia = np.insert(matriz_conteudo_copia, self.largura * id, matriz_conteudo[id, :])

      else:
        linhas_rmv = lt_linhas[diff_largura:]
        index_rmv = []
        for row in linhas_rmv:
          index_rmv.append([i for i in range(row * self.largura, (row + 1) * self.largura)])
        index_rmv = list(np.concatenate(index_rmv).flat)
        matriz_conteudo_copia = np.delete(matriz_conteudo_copia, index_rmv)

      matriz_conteudo_copia = matriz_conteudo_copia.reshape(nv_largura, self.altura)
      self.conteudo_redimensionado = matriz_conteudo_copia.flatten()

    matriz_conteudo_copia2 = matriz_conteudo_copia.copy()
    diff_altura = nv_altura - self.altura
    if diff_altura != 0:
      lt_colunas = list(range(self.altura))
      random.shuffle(lt_colunas)
      if diff_altura > 0:
        colunas_add = lt_colunas[:diff_altura]

        for id in colunas_add:
          matriz_conteudo_copia2 = np.insert(matriz_conteudo_copia2, self.altura * id, matriz_conteudo_copia[:, id])

        self.conteudo_redimensionado = matriz_conteudo_copia2

      else:
        colunas_rmv = lt_colunas[diff_altura:]
        index_rmv = []
        for col in colunas_rmv:
          index_rmv.append([i for i in range(col * self.nv_largura, (col + 1) * self.nv_largura)])
        index_rmv = list(np.concatenate(index_rmv).flat)
        matriz_conteudo_copia2 = np.delete(matriz_conteudo_copia2, index_rmv)

      self.conteudo_redimensionado = matriz_conteudo_copia2.flatten()



  def _gerar_cabecalho(self):
    return '{}\n{} {}\n {}\n'.format(self.tipo, self.largura, self.altura, self.valor_maximo)


  def salvar_imagem(self, nome, formato):
    with open('{}.{}'.format(nome, formato), 'w') as fd:
      fd.write(self.imagem)


  def salvar_imagem_rgb(self, nome, formato):
    with open('{}.{}'.format(nome+'_red', formato), 'w') as fd:
      fd.write(self.imagem_red)

    with open('{}.{}'.format(nome+'_green', formato), 'w') as fd:
      fd.write(self.imagem_green)

    with open('{}.{}'.format(nome+'_blue', formato), 'w') as fd:
      fd.write(self.imagem_blue)


  def gerar_imagem(self, colorido = False, random = True, compactado = False, redimensionar = False, gerar_rgb = False, media_escala_cinza = False, media_rgb=False):

    if random:
      cabecalho = self._gerar_cabecalho()
      matriz = '\n'.join([' '.join([str(random.randint(0, self.valor_maximo)) for _ in range(self.altura * 3 if colorido else self.altura)]) for _ in range(self.largura)])
      self.imagem = cabecalho + matriz

    elif compactado:
      self.imagem = '{} \n{} {} \n{} \n{}'.format(
          self.tipo, self.nova_largura, self.nova_altura, self.valor_maximo,
          '\n'.join([str(i) + ' ' for i in self.conteudo_compactado]))

    elif redimensionar:
      self.imagem = '{}\n{} {}\n{}'.format(
          self.tipo, self.nv_largura, self.nv_altura,
          '\n'.join([str(i) for i in self.conteudo_redimensionado.tolist()]))

    elif gerar_rgb:
      self.imagem_red = '{}\n{} {}\n{}\n{}'.format(
          self.tipo, self.largura, self.altura, self.valor_maximo,
          '\n'.join([str(i) for i in self.lista_rgb['red'].tolist()]))
      self.imagem_green = '{}\n{} {}\n{}\n{}'.format(
          self.tipo, self.largura, self.altura, self.valor_maximo,
          '\n'.join([str(i) for i in self.lista_rgb['green'].tolist()]))
      self.imagem_blue = '{}\n{} {}\n{}\n{}'.format(
          self.tipo, self.largura, self.altura, self.valor_maximo,
          '\n'.join([str(i) for i in self.lista_rgb['blue'].tolist()]))

    elif media_escala_cinza:
      self.imagem = '{}\n{} {}\n{}\n{}'.format(
          self.tipo, self.largura, self.altura, self.valor_maximo,
          '\n'.join([str(i) for i in self.conteudo_agrupado.tolist()]))

    elif media_rgb:
      self.imagem = '{}\n{} {}\n{}\n{}'.format(
          self.tipo, self.largura, self.altura, self.valor_maximo,
          '\n'.join([str(i) for i in self.conteudo_media_rgb.tolist()]))

    else:
      self.imagem = '{}\n{} {}\n{}'.format(
          self.tipo, self.largura, self.altura,
          '\n'.join([str(i) for i in self.conteudo.tolist()]))


In [ ]:
t1 = ImagemBPM()
t1.extrair_imagem("/content/Entrada_EscalaCinza.txt")
t1.compactar(4)
t1.gerar_imagem(random=False, compactado=True)
t1.salvar_imagem('img_cinza_comp_min', 'pgm')
print(t1.nova_altura, t1.nova_largura, t1.tipo, len(t1.conteudo_compactado))
#len(t1.conteudo_compactado)

#assert t1.altura % 11 == 0, 'O fator de compactação precisa ser múltiplo da altura'

In [ ]:
t1 = ImagemBPM()
t1.extrair_imagem("/content/Entrada_EscalaCinza.txt")
t1.redimensionar(480, 320)
#t1.compactar(4)
t1.gerar_imagem(random=False, compactado=False, redimensionar=True)
t1.salvar_imagem('img_cinza_480_320', 'pgm')
#print(t1.nova_altura, t1.nova_largura, t1.tipo, len(t1.conteudo_compactado))
#len(t1.conteudo_compactado)

#assert t1.altura % 11 == 0, 'O fator de compactação precisa ser múltiplo da altura'

In [ ]:
t1 = ImagemBPM()
t1.extrair_imagem("/content/Entrada_EscalaCinza.txt")
t1.redimensionar(1280, 720)
t1.gerar_imagem(random=False, compactado=False, redimensionar=True)
t1.salvar_imagem('img_cinza_1280_720', 'pgm')

In [ ]:
t1 = ImagemBPM()
t1.extrair_imagem("/content/Entrada_EscalaCinza.txt")
t1.redimensionar(1920, 1080)
t1.gerar_imagem(random=False, compactado=False, redimensionar=True)
t1.salvar_imagem('img_cinza_1920_1080', 'pgm')

In [ ]:
t1 = ImagemBPM(colorido=True)
t1.extrair_imagem("/content/Entrada_RGB.txt")
t1.compactar(4)
t1.gerar_imagem(random=False, compactado=True)
t1.salvar_imagem('img_rgb_comp_media', 'pgm')
print(t1.nova_altura, t1.nova_largura, t1.tipo, len(t1.conteudo_compactado))
#len(t1.conteudo_compactado)

#assert t1.altura % 11 == 0, 'O fator de compactação precisa ser múltiplo da altura'

In [ ]:
t1 = ImagemBPM(colorido=True)
t1.extrair_imagem("/content/Entrada_RGB.txt")
t1.redimensionar(799, 799)
t1.gerar_imagem(random=False, compactado=False, redimensionar=True)
t1.salvar_imagem('img_rgb_480_320', 'pgm')

## Segmentacao RGB

In [ ]:
t1 = ImagemBPM(colorido=True)
t1.extrair_imagem("/content/Fig1.txt")
t1.gerar_imagem(random=False, gerar_rgb=True)
t1.salvar_imagem_rgb('fig1_segmentacao_rgb', 'pgm')

In [ ]:
t1 = ImagemBPM(colorido=True)
t1.extrair_imagem("/content/Fig1.txt", default_rgb=255)
t1.gerar_imagem(random=False, gerar_rgb=True)
t1.salvar_imagem_rgb('fig1_segmentacao_rgb_max', 'pgm')

In [ ]:
t1 = ImagemBPM(colorido=True)
t1.extrair_imagem("/content/Fig1.txt")
t1.gerar_media_pixels(escala_cinza=True)
t1.gerar_imagem(random=False, media_escala_cinza=True)
t1.salvar_imagem('fig1_media_escala_cinza', 'pgm')

In [ ]:
t1 = ImagemBPM(colorido=True)
t1.extrair_imagem("/content/Fig1.txt")
t1.gerar_media_pixels_rgb()
t1.gerar_imagem(random=False, media_rgb=True)
t1.salvar_imagem('fig1_media_rgb', 'pgm')

In [ ]:
t1 = ImagemBPM(colorido=True)
t1.extrair_imagem("/content/Fig4.txt")
t1.gerar_imagem(random=False, gerar_rgb=True)
t1.salvar_imagem_rgb('Fig4_segmentacao_rgb', 'pgm')

In [ ]:
t1 = ImagemBPM(colorido=True)
t1.extrair_imagem("/content/Fig4.txt", default_rgb=255)
t1.gerar_imagem(random=False, gerar_rgb=True)
t1.salvar_imagem_rgb('Fig4_segmentacao_rgb_max', 'pgm')

In [ ]:
t1 = ImagemBPM(colorido=True)
t1.extrair_imagem("/content/Fig4.txt")
t1.gerar_media_pixels(escala_cinza=True)
t1.gerar_imagem(random=False, media_escala_cinza=True)
t1.salvar_imagem('Fig4_media_escala_cinza', 'pgm')

In [ ]:
t1 = ImagemBPM(colorido=True)
t1.extrair_imagem("/content/Fig4.txt")
t1.gerar_media_pixels_rgb()
t1.gerar_imagem(random=False, media_rgb=True)
t1.salvar_imagem('Fig4_media_rgb', 'pgm')